In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
A01_output = pd.read_csv("A01_output.csv")
A01_pd_info = pd.read_csv("A01_pd_info.csv")
A01_cust_info = pd.read_csv("A01_customer_info.csv")

A02_output = pd.read_csv("A02_output.csv")
A02_pd_info = pd.read_csv("A02_pd_info.csv")
A02_cust_info = pd.read_csv("A02_customer_info.csv")

A03_output = pd.read_csv("A03_output.csv")
A03_pd_info = pd.read_csv("A03_pd_info.csv")
A03_cust_info = pd.read_csv("A03_customer_info.csv")

A04_output = pd.read_csv("A04_output.csv")
A04_pd_info = pd.read_csv("A04_pd_info.csv")
A04_cust_info = pd.read_csv("A04_customer_info.csv")

A05_output = pd.read_csv("A05_output.csv")
A05_pd_info = pd.read_csv("A05_pd_info.csv")
A05_cust_info = pd.read_csv("A05_customer_info.csv")

A06_output = pd.read_csv("A06_output.csv")
A06_pd_info = pd.read_csv("A06_pd_info.csv")
A06_cust_info = pd.read_csv("A06_customer_info.csv")

# 모델 활용 코드

In [3]:
As = []
for i in tqdm_notebook(range(1,7)):
    As.append(pd.read_csv("A0"+str(i)+".csv",encoding="utf-8-sig"))
AA = pd.concat(As)

  0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
cust_baseline = pd.DataFrame(AA["cust"].unique(), columns=["cust"])
cust_baseline

,cust
0,M430112881
1,M646853852
2,M669384418
3,M886923095
4,M707991383
...,...
26912,M423644302
26913,M316481461
26914,M765148247
26915,M498069014


## 활용 방안 1 : 개인화 상품 광고

### 고객 별 유통사 구매 비율 생성

In [6]:
A_cop_count = AA.value_counts(["cust","cop_c"]).reset_index().rename(columns={0:"count"})
A_cop_count

,cust,cop_c,count
0,M057015266,A04,12476
1,M919374790,A04,10047
2,M763326819,A03,2474
3,M287960590,A02,2409
4,M124357021,A04,2143
...,...,...,...
67997,M257517537,A06,1
67998,M257536546,A02,1
67999,M257547604,A06,1
68000,M903676514,A04,1


In [7]:
A_cop_p = pd.merge(A_cop_count, A_cop_count.groupby("cust",as_index=False).sum()[["cust","count"]].rename(columns={"count":"cust_sum"}),how="left",on="cust")
A_cop_p

,cust,cop_c,count,cust_sum
0,M057015266,A04,12476,12486
1,M919374790,A04,10047,10049
2,M763326819,A03,2474,2524
3,M287960590,A02,2409,2564
4,M124357021,A04,2143,2143
...,...,...,...,...
67997,M257517537,A06,1,41
67998,M257536546,A02,1,13
67999,M257547604,A06,1,343
68000,M903676514,A04,1,146


In [8]:
A_cop_p["cop_per"] = A_cop_p["count"]/ A_cop_p["cust_sum"]
A_cop_p.drop(["count","cust_sum"],axis=1,inplace=True)
A_cop_p

,cust,cop_c,cop_per
0,M057015266,A04,0.999199
1,M919374790,A04,0.999801
2,M763326819,A03,0.980190
3,M287960590,A02,0.939548
4,M124357021,A04,1.000000
...,...,...,...
67997,M257517537,A06,0.024390
67998,M257536546,A02,0.076923
67999,M257547604,A06,0.002915
68000,M903676514,A04,0.006849


### 함수 생성

In [89]:
def application1(cust):
    cop = A_cop_p[A_cop_p["cust"]==cust].sample(weights = "cop_per")["cop_c"].values[0]
    if cop == "A01":
        cust_label = A01_cust_info[A01_cust_info["cust"]==cust]["label"].values[0]
        tmp = A01_output.loc[A01_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A01_pd_info.loc[A01_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A02":
        cust_label = A02_cust_info[A02_cust_info["cust"]==cust]["label"].values[0]
        tmp = A02_output.loc[A02_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A02_pd_info.loc[A02_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A03":
        cust_label = A03_cust_info[A03_cust_info["cust"]==cust]["label"].values[0]
        tmp = A03_output.loc[A03_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A03_pd_info.loc[A03_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A04":
        cust_label = A04_cust_info[A04_cust_info["cust"]==cust]["label"].values[0]
        tmp = A04_output.loc[A04_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A04_pd_info.loc[A04_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A05":
        cust_label = A05_cust_info[A05_cust_info["cust"]==cust]["label"].values[0]
        tmp = A05_output.loc[A05_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A05_pd_info.loc[A05_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A06":
        cust_label = A06_cust_info[A06_cust_info["cust"]==cust]["label"].values[0]
        tmp = A06_output.loc[A06_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A06_pd_info.loc[A06_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    
    return reco_pd

In [90]:
app1 = cust_baseline.copy()

recommend_pds=[]

for i in tqdm_notebook(app1["cust"]):
    recommend_pds.append(application1(i))

  0%|          | 0/26917 [00:00<?, ?it/s]

In [91]:
app1["pd_c"] = recommend_pds
app1

,cust,pd_c
0,M430112881,PD1484
1,M646853852,PD1183
2,M669384418,PD1331
3,M886923095,PD1683
4,M707991383,PD1769
...,...,...
26912,M423644302,PD1531
26913,M316481461,PD1560
26914,M765148247,PD0186
26915,M498069014,PD0669


In [92]:
#상품 정포
pds = pd.read_csv("LPOINT_BIG_COMP_04_PD_CLAC.csv")
pds

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,PD0001,소파,가구,거실가구
1,PD0002,스툴/리빙의자,가구,거실가구
2,PD0003,탁자,가구,거실가구
3,PD0004,장식장/진열장,가구,거실가구
4,PD0005,기타가구,가구,기타가구
...,...,...,...,...
1928,PD1929,여성향수,화장품/뷰티케어,향수
1929,PD1930,남성향수,화장품/뷰티케어,향수
1930,PD1931,남녀공용향수,화장품/뷰티케어,향수
1931,PD1932,남성향수세트,화장품/뷰티케어,향수


In [93]:
#상품 이름, 대분류, 중분류 추가
app1_1 = pd.merge(app1,pds,how="left",on="pd_c")
app1_1

,cust,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,M430112881,PD1484,종량제봉투,청소/세탁/욕실용품,청소용품
1,M646853852,PD1183,콜라,음료,탄산음료
2,M669384418,PD1331,수입맥주,주류,맥주
3,M886923095,PD1683,한식,테넌트/음식점,식당
4,M707991383,PD1769,여성가방액세서리,패션잡화,여성가방
...,...,...,...,...,...
26912,M423644302,PD1531,닭가슴살,축산물,닭고기류
26913,M316481461,PD1560,한우선물세트,축산물,축산선물세트
26914,M765148247,PD0186,기타국산과일류,과일,국산과일
26915,M498069014,PD0669,남아잠옷,속옷/양말/홈웨어,홈웨어


## 활용 방안 1 예시

In [27]:
cust = "M430112881"

In [28]:
A_cop_p[A_cop_p["cust"]==cust]

,cust,cop_c,cop_per
4070,M430112881,A03,0.638554
11138,M430112881,A02,0.255422
21564,M430112881,A01,0.098795
54374,M430112881,A04,0.007229


In [29]:
cop = A_cop_p[A_cop_p["cust"]==cust].sample(weights = "cop_per")["cop_c"].values[0]
cop

'A03'

In [72]:
cust_label = A03_cust_info[A03_cust_info["cust"]==cust]["label"].values[0]
tmp = A03_output.loc[A03_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
tmp

,cust,pd_c,ages,ma_fem_dv,pred_score
3259132,4038,666,50.0,1,2.898957
3258732,4038,200,50.0,1,2.894274
3259034,4038,541,50.0,1,2.829167
3258730,4038,198,50.0,1,2.759025
3259036,4038,544,50.0,1,2.754929
3258731,4038,199,50.0,1,2.721852
3258709,4038,166,50.0,1,2.703260
3258938,4038,425,50.0,1,2.667021
3258640,4038,81,50.0,1,2.663144
3258973,4038,473,50.0,1,2.614424


In [77]:
pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
pd_label

448

In [78]:
tmp[tmp["pd_c"]==pd_label]

,cust,pd_c,ages,ma_fem_dv,pred_score
3258951,4038,448,50.0,1,2.489838


In [79]:
#PD 코드
reco_pd = A03_pd_info.loc[A03_pd_info["label"] == pd_label,"pd_c"].values[0]
reco_pd

'PD1179'

In [80]:
pds[pds["pd_c"] == reco_pd]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
1178,PD1179,일반차음료,음료,차음료


## 활용 방안 2 : 구독 서비스

In [94]:
def application2(cust, cop):
    if cop == "A01":
        cust_label = A01_cust_info[A01_cust_info["cust"]==cust]["label"].values[0]
        tmp = A01_output.loc[A01_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A01_pd_info.loc[A01_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A02":
        cust_label = A02_cust_info[A02_cust_info["cust"]==cust]["label"].values[0]
        tmp = A02_output.loc[A02_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A02_pd_info.loc[A02_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A03":
        cust_label = A03_cust_info[A03_cust_info["cust"]==cust]["label"].values[0]
        tmp = A03_output.loc[A03_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A03_pd_info.loc[A03_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A04":
        cust_label = A04_cust_info[A04_cust_info["cust"]==cust]["label"].values[0]
        tmp = A04_output.loc[A04_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A04_pd_info.loc[A04_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A05":
        cust_label = A05_cust_info[A05_cust_info["cust"]==cust]["label"].values[0]
        tmp = A05_output.loc[A05_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A05_pd_info.loc[A05_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    elif cop == "A06":
        cust_label = A06_cust_info[A06_cust_info["cust"]==cust]["label"].values[0]
        tmp = A06_output.loc[A06_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]
        tmp.loc[tmp["pred_score"] < 0,"pred_score"] = 0
        pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
        
        reco_pd = A06_pd_info.loc[A06_pd_info["label"] == pd_label,"pd_c"].values[0]
        
    
    return reco_pd

## 활용 방안 2 예시

In [81]:
cust = "M430112881"

In [82]:
cop = "A02"

In [83]:
cust_label = A03_cust_info[A03_cust_info["cust"]==cust]["label"].values[0]
tmp = A03_output.loc[A03_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)[0:30]

In [84]:
tmp

,cust,pd_c,ages,ma_fem_dv,pred_score
3259132,4038,666,50.0,1,2.898957
3258732,4038,200,50.0,1,2.894274
3259034,4038,541,50.0,1,2.829167
3258730,4038,198,50.0,1,2.759025
3259036,4038,544,50.0,1,2.754929
3258731,4038,199,50.0,1,2.721852
3258709,4038,166,50.0,1,2.703260
3258938,4038,425,50.0,1,2.667021
3258640,4038,81,50.0,1,2.663144
3258973,4038,473,50.0,1,2.614424


In [85]:
pd_label = tmp.sample(weights = "pred_score")["pd_c"].values[0]
pd_label

425

In [86]:
tmp[tmp["pd_c"]==pd_label]

,cust,pd_c,ages,ma_fem_dv,pred_score
3258938,4038,425,50.0,1,2.667021


In [87]:
#PD 코드
reco_pd = A03_pd_info.loc[A03_pd_info["label"] == pd_label,"pd_c"].values[0]
reco_pd

'PD1153'

In [88]:
pds[pds["pd_c"] == reco_pd]

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
1152,PD1153,일반요구르트,유제품,요구르트


## 활용 방안 3 : 스탬프 미션

In [94]:
pds = pd.read_csv("LPOINT_BIG_COMP_04_PD_CLAC.csv")

#스탬프 리스트, 스탬프 상품 목록 반환
def application3(cust, cop):
    
    if cop == "A01":
        cust_label = A01_cust_info[A01_cust_info["cust"]==cust]["label"].values[0]
        tmp = A01_output.loc[A01_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A01_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    elif cop == "A02":
        cust_label = A02_cust_info[A02_cust_info["cust"]==cust]["label"].values[0]
        tmp = A02_output.loc[A02_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A02_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    elif cop == "A03":
        cust_label = A03_cust_info[A03_cust_info["cust"]==cust]["label"].values[0]
        tmp = A03_output.loc[A03_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A03_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    elif cop == "A04":
        cust_label = A04_cust_info[A04_cust_info["cust"]==cust]["label"].values[0]
        tmp = A04_output.loc[A04_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A04_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    elif cop == "A05":
        cust_label = A05_cust_info[A05_cust_info["cust"]==cust]["label"].values[0]
        tmp = A05_output.loc[A05_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A05_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    elif cop == "A06":
        cust_label = A06_cust_info[A06_cust_info["cust"]==cust]["label"].values[0]
        tmp = A06_output.loc[A06_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
        tmp = tmp.rename(columns={"pd_c":"label"})
        tmp1 = pd.merge(tmp,A06_pd_info,how="left",on="label")
        tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
        tmp3 = tmp2.groupby("clac_mcls_nm",as_index=False).mean("pred_score")[["clac_mcls_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
        stamp_list =  list(tmp3["clac_mcls_nm"])
        stamp_dict = {}
        for i in stamp_list:
            pd_list = tmp2.loc[tmp2["clac_mcls_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)
            stamp_dict[i] = pd_list
        
    
    return stamp_list, stamp_dict

## 활용 방안 3 예시

In [112]:
cust_label = A04_cust_info[A01_cust_info["cust"]=="M430112881"]["label"].values[0]
cust_label

8679

In [113]:
tmp = A04_output.loc[A04_output["cust"]==cust_label,:].sort_values("pred_score",ascending=False)
tmp = tmp.rename(columns={"pd_c":"label"})
tmp

,cust,label,ages,ma_fem_dv,pred_score
3690014,8679,120,50.0,1,2.685902
3690015,8679,121,50.0,1,2.651316
3690018,8679,124,50.0,1,2.561704
3690082,8679,188,50.0,1,2.524580
3690017,8679,123,50.0,1,2.310347
...,...,...,...,...,...
3690291,8679,399,50.0,1,0.826116
3690289,8679,397,50.0,1,0.809842
3690324,8679,432,50.0,1,0.765537
3690317,8679,425,50.0,1,0.762858


In [114]:
tmp1 = pd.merge(tmp,A04_pd_info,how="left",on="label")
tmp1

,cust,label,ages,ma_fem_dv,pred_score,pd_c
0,8679,120,50.0,1,2.685902,PD0380
1,8679,121,50.0,1,2.651316,PD0381
2,8679,124,50.0,1,2.561704,PD0384
3,8679,188,50.0,1,2.524580,PD0529
4,8679,123,50.0,1,2.310347,PD0383
...,...,...,...,...,...,...
428,8679,399,50.0,1,0.826116,PD1828
429,8679,397,50.0,1,0.809842,PD1797
430,8679,432,50.0,1,0.765537,PD1926
431,8679,425,50.0,1,0.762858,PD1912


In [115]:
#상품 정포
pds = pd.read_csv("LPOINT_BIG_COMP_04_PD_CLAC.csv")
pds

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,PD0001,소파,가구,거실가구
1,PD0002,스툴/리빙의자,가구,거실가구
2,PD0003,탁자,가구,거실가구
3,PD0004,장식장/진열장,가구,거실가구
4,PD0005,기타가구,가구,기타가구
...,...,...,...,...
1928,PD1929,여성향수,화장품/뷰티케어,향수
1929,PD1930,남성향수,화장품/뷰티케어,향수
1930,PD1931,남녀공용향수,화장품/뷰티케어,향수
1931,PD1932,남성향수세트,화장품/뷰티케어,향수


In [116]:
tmp2 = pd.merge(tmp1,pds,how="left",on="pd_c")
tmp2

,cust,label,ages,ma_fem_dv,pred_score,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,8679,120,50.0,1,2.685902,PD0380,수입담배,담배,일반담배
1,8679,121,50.0,1,2.651316,PD0381,국산담배,담배,일반담배
2,8679,124,50.0,1,2.561704,PD0384,전자담배용기기/악세서리,담배,전자담배
3,8679,188,50.0,1,2.524580,PD0529,기타생활서비스,생활/렌탈서비스,생활서비스
4,8679,123,50.0,1,2.310347,PD0383,전자담배용리필,담배,전자담배
...,...,...,...,...,...,...,...,...,...
428,8679,399,50.0,1,0.826116,PD1828,여성헤어액세서리,패션잡화,헤어액세서리
429,8679,397,50.0,1,0.809842,PD1797,우비,패션잡화,우산/양산류
430,8679,432,50.0,1,0.765537,PD1926,립밤/립스크럽,화장품/뷰티케어,스킨케어
431,8679,425,50.0,1,0.762858,PD1912,면봉/화장솜,화장품/뷰티케어,미용소품


In [117]:
tmp3 = tmp2.groupby("clac_hlv_nm",as_index=False).mean("pred_score")[["clac_hlv_nm","pred_score"]].sort_values("pred_score",ascending=False)[0:5]
tmp3

,clac_hlv_nm,pred_score
11,담배,2.406099
7,기타(비상품),2.028876
17,생활/렌탈서비스,1.999664
33,조리식품,1.961178
30,음료,1.898021


In [118]:
stamp_list =  list(tmp3["clac_hlv_nm"])
stamp_list

['담배', '기타(비상품)', '생활/렌탈서비스', '조리식품', '음료']

In [119]:
stamp_dict = {}
for i in stamp_list:
    pd_list = tmp2.loc[tmp2["clac_hlv_nm"] == i,["pd_c","pd_nm","pred_score"]].sort_values("pred_score",ascending=False)

    stamp_dict[i] = pd_list

In [120]:
stamp_dict[stamp_list[0]]

,pd_c,pd_nm,pred_score
0,PD0380,수입담배,2.685902
1,PD0381,국산담배,2.651316
2,PD0384,전자담배용기기/악세서리,2.561704
4,PD0383,전자담배용리필,2.310347
90,PD0382,담배관련용품,1.821227


In [121]:
stamp_dict[stamp_list[1]]

,pd_c,pd_nm,pred_score
24,PD0282,봉투보증금,2.028876


In [122]:
stamp_dict[stamp_list[2]]

,pd_c,pd_nm,pred_score
3,PD0529,기타생활서비스,2.524580
49,PD0521,택배,1.916140
175,PD0522,복권판매,1.558274


In [123]:
stamp_dict[stamp_list[3]]

,pd_c,pd_nm,pred_score
8,PD1245,도시락,2.176865
11,PD1243,삼각김밥,2.119776
13,PD1261,치킨류,2.106977
18,PD1247,샌드위치,2.067512
23,PD1246,샐러드,2.038820
25,PD1248,햄버거,2.018593
29,PD1241,김밥,1.994317
33,PD1269,기타온장조리,1.973014
35,PD1252,일반떡,1.972702
89,PD1249,기타냉장조리,1.823697


In [124]:
stamp_dict[stamp_list[4]]

,pd_c,pd_nm,pred_score
6,PD1178,파우치/즉석음료,2.216861
7,PD1181,커피음료,2.197330
15,PD1166,비타민/에너지음료,2.092733
20,PD1183,콜라,2.061656
21,PD1165,스포츠음료,2.056653
27,PD1173,생수,2.003981
37,PD1179,일반차음료,1.967129
39,PD1174,탄산수,1.955084
47,PD1161,과일음료,1.923044
72,PD1171,일반두유,1.851306
